In [1]:
import pyrosetta_help as ph
from pathlib import Path
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from collections import Counter
prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring
pr_options: ModuleType = pyrosetta.rosetta.basic.options


logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                                     ex1=None,
                                                                     ex2=None,
                                                                     # mute='all',
                                                                     ignore_unrecognized_res=True,
                                                                     load_PDB_components=False,
                                                                     ignore_waters=True)
                                 )


def scan(pose, name, chains, interfaces, offset=1):
    aas = {
        "A": "Ala",
        "R": "Arg",
        "N": "Asn",
        "D": "Asp",
        "C": "Cys",
        "E": "Glu",
        "Q": "Gln",
        "G": "Gly",
        "H": "His",
        "I": "Ile",
        "L": "Leu",
        "K": "Lys",
        "M": "Met",
        "F": "Phe",
        "P": "Pro",
        "S": "Ser",
        "T": "Thr",
        "W": "Trp",
        "Y": "Tyr",
        "V": "Val"
    }
    model = ph.MutantScorer(pose, modelname=name)
    model.scorefxn = pyrosetta.create_score_function('ref2015')
    model.strict_about_starting_residue = True
    mutations = [f'{aas[r]}{i+offset}{t}' for i, r in enumerate(pose.chain_sequence(2)) for t in aas.values()]
    data = model.score_mutations(mutations,
                                 chains=chains,
                                 interfaces=interfaces,  #
                                 preminimize=False,
                                 distance=8,
                                 cycles=5)
    import pandas as pd
    scores = pd.DataFrame(data)
    ph.extend_scores(scores)
    return scores

/data/xchem-fragalysis/mferla/waconda/envs/compchem/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [ ]:
filename = 'collage.weak_con.pdb'
pose = pyrosetta.pose_from_file(filename)
open_cores = scan(pose, 'open', chains='B', offset=16, interfaces=(('isolated', 'A_B'),))
open_scores.to_csv('open_mutational_scan.csv')

filename = 'template2.pdb'
closed_pose = pyrosetta.pose_from_file(filename)
closed_scores = scan(pose, 'ref', chains='B', offset=16, interfaces=(('isolated', 'A_B'),))
closed_scores.to_csv('open_mutational_scan.csv')